In [1]:
import math
import time
import pandas as pd
import numpy as np
import numpy.linalg as la

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor

In [2]:
def load_data(path):
    df = pd.read_csv(path)
    data = df.drop(columns=['Unnamed: 0'])
    
    return data

In [3]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate)
    train_data = data[0:train_size]
    test_data = data[train_size:time_len]
    
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0 : seq_len])
        trainY.append(a[seq_len : seq_len + pre_len])
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
        
    return trainX, trainY, testX, testY

In [4]:
def getTestY(data):
    tmp_scaler = StandardScaler()
    tmp_data = tmp_scaler.fit_transform(data)
    tmp_data = tmp_scaler.inverse_transform(tmp_data)
    _, _, _, testY = preprocess_data(tmp_data, data.shape[0], train_rate, seq_len, pre_len)
    testY = np.array(testY)
    testY = np.reshape(testY, [-1, pre_len])
    
    return testY

In [5]:
 def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    mape = mean_absolute_percentage_error(a, b)
    F_norm = la.norm(a-b)/la.norm(a)
    
    return rmse, mae, mape, 1-F_norm

In [6]:
def predictSVR(data, testY):
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    
    rmses, maes, mapes, accs = [], [], [], []
    for i in range(data.shape[1]):
        print('Node', i)
        start = time.time()
        a = data[:, i]
        aX, aY, tX, tY = preprocess_data(a, data.shape[0], train_rate, seq_len, pre_len)
        
        aX = np.array(aX)
        aX = np.reshape(aX, [-1, seq_len])
        aY = np.array(aY)
        aY = np.reshape(aY, [-1, pre_len])

        tX = np.array(tX)
        tX = np.reshape(tX, [-1, seq_len])
        tY = np.array(tY)
        tY = np.reshape(tY, [-1, pre_len])
        
        print('(', aX.shape, aY.shape, tX.shape, tY.shape, round(aX.mean(),3), ')')
        reg = MultiOutputRegressor(SVR(kernel='linear'))
        reg.fit(aX, aY)
        pred = reg.predict(tX)
        
        mean = scaler.mean_[i]
        std = np.sqrt(scaler.var_[i])
        pred = pred*std + mean
        tY = tY*std + mean
        if i==data.shape[1]-1: tY = testY[tY.shape[0]*i:]
        else: tY = testY[tY.shape[0] * i:tY.shape[0] * (i+1)]
        
        rmse, mae, mape, acc = evaluation(tY, pred)
        rmses.append(rmse)
        maes.append(mae)
        mapes.append(mape)
        accs.append(acc)
        
        print('(', rmse, mae, mape, acc, time.time() - start)
    
    print('RMSE: ' + str(np.mean(rmses)) + ', MAE: ' + str(np.mean(maes)) + ', MAPE: ' + str(np.mean(mapes)) + ', ACC: ' + str(np.mean(accs)))

In [7]:
train_rate = 0.8
seq_len = 12
pre_len = 3

In [8]:
path_0 = '../Data/Gangnam/speed_gangnam_0.csv'
path_5 = '../Data/Gangnam/speed_gangnam_5.csv'
path_10 = '../Data/Gangnam/speed_gangnam_10.csv'
path_20 = '../Data/Gangnam/speed_gangnam_20.csv'

In [9]:
masterData = load_data(path_0)
print(masterData.shape, (masterData == 0).sum().sum())
testY = getTestY(masterData)
testY.shape, (testY == 0).sum().sum()

(2880, 506) 0


((283866, 3), 0)

### Unobserved Nodes = 10%

In [10]:
data = load_data(path_10)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(2880, 506) 144000
Node 0
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.004 )
( 10.781828171266614 7.573932456364159 0.2484894316434689 0.6620980869270952 1.2875616550445557
Node 1
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.031 )
( 13.126655044288654 9.795921929793481 0.3222244009159591 0.5867180802152745 1.0771195888519287
Node 2
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.093 )
( 12.279780691253848 9.564082905786846 0.34265111977234275 0.6197629279439404 1.1838364601135254
Node 3
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 32.38985549119267 30.72692216280451 1.0 0.0 0.00598454475402832
Node 4
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.006 )
( 16.122897179839924 12.602131479262946 0.3787103007378827 0.5014182730218815 1.2017884254455566
Node 5
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.021 )
( 13.373843740673136 9.735110775224106 0.28113030800758204 0.5915369482389541 1.444138765335083
Node 6
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 32.788671098512324 31.060540701128

( 13.468949357237356 10.8161982939286 0.420814051926681 0.5918644013072845 1.1928114891052246
Node 57
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.006 )
( 11.834811313951734 8.725826497322426 0.293949924761065 0.6451841560350666 1.3214950561523438
Node 58
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.037 )
( 15.764479114727372 13.383375802028167 0.5298535704297828 0.5238883642010567 1.1419637203216553
Node 59
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.001 )
( 12.162991301565818 9.40163837927386 0.3582647298051235 0.6286957087150749 1.4071967601776123
Node 60
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.051 )
( 11.550575274753506 8.87162237471066 0.3301421232858669 0.6527611754125552 1.2008275985717773
Node 61
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.061 )
( 16.051197627757006 12.717286162641926 0.3735267540129856 0.5134088057190913 1.1000523567199707
Node 62
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.003 )
( 11.369187868974405 8.706465603738152 0.32676960234451496 0.6556448085836

( 10.244333004378499 7.90430839253978 0.3858797057535546 0.6206537378026844 1.6954686641693115
Node 111
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.017 )
( 9.67219453623295 7.061338699775679 0.3098821837384649 0.6374647831750481 1.427184820175171
Node 112
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.025 )
( 9.707627885255587 7.29563697682095 0.33366142611271005 0.6387521165508248 1.4521198272705078
Node 113
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.001 )
( 11.041708417363008 8.44687539905834 0.3619500760772539 0.5893700631148863 1.3035156726837158
Node 114
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.038 )
( 9.100325146159705 6.722204843098656 0.3407676839861516 0.6552893065694947 1.817143201828003
Node 115
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.015 )
( 9.98373063189567 7.437331325438997 0.35212129221360994 0.6214675014567042 1.8211331367492676
Node 116
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.049 )
( 11.61647868010619 9.456920255429239 0.4885462877288802 0.5661443580773281 1.4

( 12.594674453668922 9.968228742987797 0.5711615120045282 0.5154644483794386 1.5029826164245605
Node 166
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.061 )
( 18.278060837152047 15.118355982136999 0.8376698181637585 0.2988028275353033 1.2945401668548584
Node 167
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.057 )
( 17.822336653022287 14.425188427823826 0.7890256842228606 0.3199676018601938 1.2985296249389648
Node 168
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.006 )
( 28.979326433831822 27.280825463987412 1.4454034925357593 -0.09919672417811753 1.849060297012329
Node 169
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.007 )
( 16.013600736326374 13.182862636719962 0.6390198044619938 0.3956825869859375 1.0771193504333496
Node 170
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.019 )
( 10.425067982052022 7.701330182220082 0.35006951270252323 0.6043363059685503 1.7114274501800537
Node 171
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.005 )
( 12.67271737933375 10.427874433102412 0.5643341548838868 0.515

( 11.985590444392216 9.706842153156972 0.43767311807069137 0.5852757944747227 1.6515872478485107
Node 220
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.008 )
( 12.98961253157194 10.432276717606834 0.4663475078392145 0.5577956995701269 1.5428743362426758
Node 221
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.013 )
( 10.483172244360066 8.057469345229928 0.35900217206588864 0.642249590725161 1.6825032234191895
Node 222
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.016 )
( 10.322839164299683 7.3962821724124765 0.2890607256746959 0.6452912480090222 1.6216669082641602
Node 223
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.025 )
( 10.970304962885834 8.118849557506826 0.31886259158259983 0.62644719811928 1.7213985919952393
Node 224
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 29.305039918868307 27.642632204396914 1.0 0.0 0.0069828033447265625
Node 225
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.032 )
( 9.969727543586444 7.321172951418373 0.2948504561494815 0.6565722104227938 1.8041768074035645

( 13.975730153109156 10.402447443691791 0.31042534911367 0.5906931044693338 1.9807074069976807
Node 274
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.001 )
( 16.342420607133114 12.737667387077392 0.3647154304379925 0.5154781020175625 1.5169451236724854
Node 275
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.025 )
( 16.172324918598015 12.342099139630415 0.3507412264348284 0.5260854684823899 1.6854944229125977
Node 276
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 34.12592395611563 32.25314913844325 1.0 0.0 0.006982564926147461
Node 277
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.129 )
( 17.933089046743817 14.497608005853332 0.4099950350023847 0.4713003629924106 1.8031792640686035
Node 278
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.051 )
( 14.771719514673649 11.091438375925947 0.3138899143976409 0.5691785447633337 1.9408128261566162
Node 279
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.043 )
( 15.934670313828 12.230185323046223 0.3488131029684278 0.5347628469762346 1.9188737869262695
No

( 16.79223496669068 13.936000622183338 0.5939653562861883 0.4958637628853222 2.251981258392334
Node 328
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 32.90384922588808 31.046785502079626 1.0 0.0 0.007979869842529297
Node 329
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.042 )
( 12.568042427448223 9.558717507410568 0.3490088983639195 0.6159457418460538 1.9677391052246094
Node 330
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 32.83958541775432 31.01385620915033 1.0 0.0 0.006983757019042969
Node 331
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.008 )
( 12.308682981425756 9.841742780600883 0.3982669005255451 0.6237024233919146 1.402251958847046
Node 332
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.025 )
( 11.832248156412403 9.17528852429208 0.34384362721626704 0.6325537096225926 1.66654634475708
Node 333
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.094 )
( 13.065711172255657 10.849064095515926 0.45074395655338 0.5972553192214669 1.700455904006958
Node 334
( (2289, 12) (2289, 3) (561, 12)

( 11.401938142864713 8.663887392338925 0.4088024099132089 0.56522133411804 1.6396191120147705
Node 382
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.035 )
( 13.414592921632721 10.286789276711701 0.47075317173263254 0.49464339378058075 1.3035483360290527
Node 383
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.029 )
( 10.205734426613516 7.775988738310748 0.4014405866349897 0.6099805668550724 1.5408506393432617
Node 384
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.053 )
( 10.758464647078133 8.05343506317917 0.4018090339623262 0.5845363901215566 1.4960012435913086
Node 385
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 26.180129059570312 24.210338680926913 1.0 0.0 0.009974002838134766
Node 386
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.001 )
( 10.813728447186767 8.742413314922715 0.4801706516177584 0.5863067684817183 1.8261191844940186
Node 387
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.031 )
( 12.411933662042554 10.209808879761887 0.5560487698273235 0.5183196224790215 1.4820377826690674
N

( 10.397699082839035 7.610166567767156 0.3777164064780259 0.5904290850495683 1.2965357303619385
Node 436
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.012 )
( 11.464977733296202 8.263031983224463 0.35816031625998246 0.5450355457286644 1.452120304107666
Node 437
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.017 )
( 11.584231063861838 8.864624964732654 0.475525034818701 0.5402819294189465 1.1738626956939697
Node 438
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.021 )
( 10.346654814129693 7.921152161287417 0.4514124006400812 0.5945898374650621 1.9388179779052734
Node 439
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.022 )
( 11.45490601873954 9.425994448250435 0.5572519343772648 0.5430894598612914 1.6695387363433838
Node 440
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.048 )
( 14.986710809396056 12.917002825165754 0.7927323364793387 0.4000155439509332 1.6665446758270264
Node 441
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.015 )
( 11.64892897196353 9.705316927077716 0.5833951331867513 0.5409794327

( 17.664186238752922 15.926335451991719 0.7452419657829278 0.39453653697209423 1.5179424285888672
Node 490
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.025 )
( 14.988446144313135 13.206331544576571 0.6178429328897506 0.487047861666301 2.0368168354034424
Node 491
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.024 )
( 10.285378341233972 7.427217460689973 0.2774056146051665 0.6492334743491177 1.6426117420196533
Node 492
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.011 )
( 12.424400521882623 9.158792220904575 0.31204848026905807 0.5778995478254101 1.5777814388275146
Node 493
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 29.774040809403484 28.118253119429585 1.0 0.0 0.008976221084594727
Node 494
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.007 )
( 12.033100361557393 8.827038737156817 0.29886424034297027 0.6011397784721748 1.8680579662322998
Node 495
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.001 )
( 10.452063161467182 7.67503339992502 0.2822536377635273 0.6537036175444018 1.5557930469512

### Unobserved Nodes = 20%

In [11]:
data = load_data(path_20)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(2880, 506) 290880
Node 0
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.004 )
( 10.781828171266614 7.573932456364159 0.2484894316434689 0.6620980869270952 1.8181676864624023
Node 1
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.031 )
( 13.126655044288654 9.795921929793481 0.3222244009159591 0.5867180802152745 1.506943941116333
Node 2
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.093 )
( 12.279780691253848 9.564082905786846 0.34265111977234275 0.6197629279439404 1.7762088775634766
Node 3
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 32.38985549119267 30.72692216280451 1.0 0.0 0.011967897415161133
Node 4
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.006 )
( 16.122897179839924 12.602131479262946 0.3787103007378827 0.5014182730218815 1.7782468795776367
Node 5
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.021 )
( 13.373843740673136 9.735110775224106 0.28113030800758204 0.5915369482389541 1.8201367855072021
Node 6
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.012 )
( 11.914442884837596 8.3929684497

( 10.822135653414666 7.912108008894587 0.2668297069585068 0.6744145281617004 1.9178740978240967
Node 56
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.003 )
( 13.468949357237356 10.8161982939286 0.420814051926681 0.5918644013072845 1.570800542831421
Node 57
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.006 )
( 11.834811313951734 8.725826497322426 0.293949924761065 0.6451841560350666 1.6485939025878906
Node 58
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.037 )
( 15.764479114727372 13.383375802028167 0.5298535704297828 0.5238883642010567 1.4591009616851807
Node 59
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.001 )
( 12.162991301565818 9.40163837927386 0.3582647298051235 0.6286957087150749 1.6456012725830078
Node 60
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 33.26406627622002 31.50614973262033 1.0 0.0 0.0059850215911865234
Node 61
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.061 )
( 16.051197627757006 12.717286162641926 0.3735267540129856 0.5134088057190913 1.291548728942871
Node 62
( (

( 9.67219453623295 7.061338699775679 0.3098821837384649 0.6374647831750481 1.4790468215942383
Node 112
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 26.872483770887964 25.372174688057047 1.0 0.0 0.006033658981323242
Node 113
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.001 )
( 11.041708417363008 8.44687539905834 0.3619500760772539 0.5893700631148863 1.3802618980407715
Node 114
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.038 )
( 9.100325146159705 6.722204843098656 0.3407676839861516 0.6552893065694947 1.9787092208862305
Node 115
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.015 )
( 9.98373063189567 7.437331325438997 0.35212129221360994 0.6214675014567042 1.8769845962524414
Node 116
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.049 )
( 11.61647868010619 9.456920255429239 0.4885462877288802 0.5661443580773281 1.4630887508392334
Node 117
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.042 )
( 21.00838112418614 19.436474950899417 1.029355842976096 0.20885294937662957 1.439199447631836
Node 118


( 17.822336653022287 14.425188427823826 0.7890256842228606 0.3199676018601938 1.214751958847046
Node 168
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.006 )
( 28.979326433831822 27.280825463987412 1.4454034925357593 -0.09919672417811753 1.7882215976715088
Node 169
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.007 )
( 16.013600736326374 13.182862636719962 0.6390198044619938 0.3956825869859375 1.008303165435791
Node 170
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.019 )
( 10.425067982052022 7.701330182220082 0.35006951270252323 0.6043363059685503 1.5917799472808838
Node 171
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.005 )
( 12.67271737933375 10.427874433102412 0.5643341548838868 0.515010780991798 1.6824707984924316
Node 172
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.016 )
( 13.908897484491286 11.043485154721623 0.4947248778267815 0.4707602106536928 1.4920430183410645
Node 173
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.03 )
( 11.333794459011918 8.907815202482212 0.4725229403518824 0.561625

( 10.483172244360066 8.057469345229928 0.35900217206588864 0.642249590725161 1.6685411930084229
Node 222
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.016 )
( 10.322839164299683 7.3962821724124765 0.2890607256746959 0.6452912480090222 1.6556062698364258
Node 223
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.025 )
( 10.970304962885834 8.118849557506826 0.31886259158259983 0.62644719811928 1.6764886379241943
Node 224
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 29.305039918868307 27.642632204396914 1.0 0.0 0.004986286163330078
Node 225
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 29.030054777629285 27.47375519904932 1.0 0.0 0.0059850215911865234
Node 226
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.008 )
( 11.05525741686744 7.965563596377077 0.29033103389886233 0.6201739485406959 1.8330988883972168
Node 227
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 29.3649709237038 27.600207961972668 1.0 0.0 0.005984067916870117
Node 228
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.005 )
( 1

( 15.934670313828 12.230185323046223 0.3488131029684278 0.5347628469762346 1.7214009761810303
Node 280
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.001 )
( 12.99528183975128 9.44540410342715 0.28720146549848535 0.6195416656890367 2.0385494232177734
Node 281
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.049 )
( 15.819003547861746 12.105248970664547 0.3618815590708586 0.5433099086246969 1.8410797119140625
Node 282
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.002 )
( 15.610993703191863 12.015933504892873 0.37219669018313617 0.5529841292602398 1.453117847442627
Node 283
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.007 )
( 14.290836034786322 10.890855900684926 0.3402659074539806 0.5876022135780588 1.4182093143463135
Node 284
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.017 )
( 11.978501221384715 8.770295962626392 0.34202266124556774 0.6579412462789228 1.5398845672607422
Node 285
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 18.183669557198325 14.476546223455735 0.41744729095355243 0.4811227

( 12.749159941031985 10.349174904415305 0.43667902413505844 0.603206937125875 1.9597959518432617
Node 337
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.033 )
( 10.687277500163946 7.914912537140534 0.29499732560799674 0.6653072726145091 1.8929080963134766
Node 338
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.04 )
( 17.589341355361476 15.676891328961299 0.6582134256950835 0.4452345899861099 1.4342103004455566
Node 339
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.035 )
( 11.245642984814603 8.592268267019367 0.35478907076576993 0.6464309683150977 1.930844783782959
Node 340
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.021 )
( 10.873898400517438 8.25178487484314 0.3360797615566935 0.6571080083952743 1.7612452507019043
Node 341
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.035 )
( 11.839128074736788 9.298519395159135 0.3674745007145053 0.6240463935146529 1.6067044734954834
Node 342
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 31.49183690548268 29.944800950683298 1.0 0.0 0.0059833526611328125
No

( 11.156742766214988 7.863912692266239 0.3583658492948572 0.5672587414344232 1.829113245010376
Node 392
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.005 )
( 10.880197740029994 7.851667443945675 0.3856291487859325 0.5795091252357938 1.9727280139923096
Node 393
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.026 )
( 12.213874926963433 9.487505666596666 0.517424462031683 0.519723163412733 1.8370897769927979
Node 394
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.012 )
( 10.73798967479213 8.142912765102832 0.4813881752005102 0.5747916932950152 1.789217233657837
Node 395
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.028 )
( 11.856684870478595 8.785438945118235 0.4401621767869484 0.5331378857581706 2.0485243797302246
Node 396
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.02 )
( 10.878194394775592 8.23882342808736 0.4642799241422257 0.568550333768026 2.059497356414795
Node 397
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.016 )
( 12.896392953970679 11.092943129890465 0.6834227841667352 0.485353301184221

( 13.855671574568262 10.501195067012711 0.44646482181632363 0.45261623809390694 1.1588573455810547
Node 446
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.014 )
( 11.285196910919515 8.847947344151892 0.5088342796508849 0.554175748819508 1.6037147045135498
Node 447
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.027 )
( 10.0489981350646 7.689342531624921 0.42039755271068396 0.6055569080638543 1.6057085990905762
Node 448
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 25.401843145238153 23.56796197266785 1.0 0.0 0.005984067916870117
Node 449
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.054 )
( 10.72977321982887 7.987840669380335 0.39314794588886603 0.5795704107368953 2.1293094158172607
Node 450
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.025 )
( 9.811758201810424 7.19967070364565 0.357791734261183 0.6202073275888682 1.7274253368377686
Node 451
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.019 )
( 11.319098163585327 8.990756065212958 0.47132468510671727 0.564485045281368 1.6126456260681152
Node 4

( 10.673339018842011 8.013262715333843 0.3129271791664514 0.6556995198204658 1.5000221729278564
Node 501
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.007 )
( 10.264340684468994 7.484230502533173 0.2736504942749674 0.6692437014843444 1.7976877689361572
Node 502
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.011 )
( 11.702674056022325 8.708135004084847 0.31125189127340885 0.6284097392967609 1.361361026763916
Node 503
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 31.583163883299953 29.861354723707663 1.0 0.0 0.005984067916870117
Node 504
( (2289, 12) (2289, 3) (561, 12) (561, 3) 0.0 )
( 31.472178797550747 29.818657159833638 1.0 0.0 0.00598454475402832
Node 505
( (2289, 12) (2289, 3) (561, 12) (561, 3) -0.003 )
( 14.784093437417006 11.021093021380414 0.333662839620876 0.5373324501526043 1.6256535053253174
RMSE: 17.045028402243958, MAE: 14.386093386325268, MAPE: 0.5629433810776993, ACC: 0.42278794722077256
